In [1]:
import copy
import math
import random
import warnings
from datasets import Dataset, DatasetDict, load_dataset
from elasticsearch import Elasticsearch, helpers
import evaluate
from focal_loss.focal_loss import FocalLoss
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn.utils.class_weight import compute_class_weight
from torch import nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments)

warnings.filterwarnings("ignore")
client = Elasticsearch(
"https://host.docker.internal:9200",
verify_certs=False,
basic_auth=("elastic", "WhsHf2OVb2nxg_dmwuya"))
client.info()['version']

{'number': '8.11.1',
 'build_flavor': 'default',
 'build_type': 'docker',
 'build_hash': '6f9ff581fbcde658e6f69d6ce03050f060d1fd0c',
 'build_date': '2023-11-11T10:05:59.421038163Z',
 'build_snapshot': False,
 'lucene_version': '9.8.0',
 'minimum_wire_compatibility_version': '7.17.0',
 'minimum_index_compatibility_version': '7.0.0'}

In [2]:
#Przygotowanie potrzebnych danych
dataset = load_dataset("clarin-knext/fiqa-pl", 'corpus')['corpus'].to_pandas()
corpus_df = pd.DataFrame(dataset)
data = []
for index, row in corpus_df.iterrows():
    data.append(row)
    
dataset_questions = load_dataset("clarin-knext/fiqa-pl",'queries')['queries'].to_pandas()
qa_dataset_test = load_dataset("clarin-knext/fiqa-pl-qrels", 'corpus')['test'].to_pandas()
qa_dataset_train = load_dataset("clarin-knext/fiqa-pl-qrels", 'corpus')['train'].to_pandas()
qa_dataset_validation = load_dataset("clarin-knext/fiqa-pl-qrels", 'corpus')['validation'].to_pandas()

df_qa_ids = qa_dataset_test.append(qa_dataset_train).append(qa_dataset_validation)

dataset_questions["_id"] = dataset_questions["_id"].apply(lambda x: int(x))
corpus_df["_id"] = corpus_df["_id"].apply(lambda x: int(x))

Poniżej umieszczam dane corpusu w elasticsearch'u. Będzie mi to potrzebne do stworzenia datasetu.

In [3]:
settings = {
    "analysis": {
      "analyzer": {
        "analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": ["lowercase",
                     "morfologik_stem",
                     "lowercase"]
        }
      }
    }
  }
mappings = {
    "properties": {
      "text": {
        "type": "text",
        "analyzer": "analyzer"
      }
    }
  }

In [4]:
#client.indices.create(index="qa_index", settings=settings, mappings=mappings)
#client.indices.delete(index="qa_index")

In [5]:
# Wysyłanie danych do indeksu.
actions2 = [
    {
        "_op_type": "index",
        "_index": "qa_index",
        "_source": {
            "text": item["text"],
            "id": int(item["_id"])
        }
    }
    for item in data
]

#helpers.bulk(client, actions2)

In [6]:
# Tworzę dataset. Chcę, aby odpowiedzi wybrane do datasetu, będące błędnymi odpowiedziami, były jak najbardziej podobne do odpowiedzi poprawnych.
# Do wyboru odpowiedzi, które są niepoprawne użyję zatem FTS i elasticsearch.
SIZE = 20

def search_best_answers_es(client, index_name, question, size):
    query_body = {
        "size": size,
        "query": {
            "match": {
                "text": {
                    "query": question,
                    "analyzer": "analyzer"
                }
            }
        },
        "_source": ["id"]
    }
    response = client.search(index=index_name, body=query_body)
    return [hit["_source"]["id"] for hit in response['hits']['hits']]

def create_pairs(client, question, proper_answers, answers_dataset, size):
    elastic_answers = search_best_answers_es(client, "qa_index", question, size)
    pairs = []
    for elastic_answer in elastic_answers:
        if elastic_answer in proper_answers:
            continue
        value = corpus_df[corpus_df["_id"] == elastic_answer]["text"].values
        if len(value) == 0:
            print(elastic_answer)
        else:
            pairs.append([question, value[0], 0])
    for proper_answer in proper_answers:
        value = corpus_df[corpus_df["_id"] == proper_answer]["text"].values
        if len(value) == 0:
            print(elastic_answer)
        else:
            pairs.append([question, value[0], 1])
    random.shuffle(pairs)
    return pairs

# Wykonywanie zapytań, do elasticserach, w celu stworzenia listy pytań
import numpy as np
results = []
unique_ids = df_qa_ids["query-id"].unique()
#iterations = 3000
iterations = len(unique_ids)
for i in tqdm(range(iterations), desc="Processing"):
    id = unique_ids[i]
    question = dataset_questions[dataset_questions['_id'] == id]["text"].to_list()[0]
    res = create_pairs(client, question,  df_qa_ids[df_qa_ids["query-id"] == id]["corpus-id"].values, corpus_df, SIZE)
    results.append(res)

Processing:   0%|          | 0/6648 [00:00<?, ?it/s]

In [7]:
# Tworzę funkcje, które potem będą przydatne przy transformowaniu datasetu do wymaganego formatu
def flatten_list(list):
    res = []
    for elements in list:
        for element in elements:
            res.append(element)
    random.shuffle(res)
    return res

def to_dataframe(list):
    return pd.DataFrame(list, columns=["query", "answer", "labels"])

def data_to_dataset_dict(train, val, test):
    return DatasetDict({
        'train': Dataset.from_pandas(to_dataframe(train)),
        'validation': Dataset.from_pandas(to_dataframe(val)),
        'test': Dataset.from_pandas(to_dataframe(test))
    })

def split_data(data, train_split, val_split, test_split):
    copied_list = copy.deepcopy(data)
    random.shuffle(copied_list)
    train_idx = int(len(copied_list) * train_split)
    val_idx = int(len(copied_list) * (train_split + val_split))

    # Split the data
    train_data = copied_list[:train_idx]
    val_data = copied_list[train_idx:val_idx]
    test_data = copied_list[val_idx:]
    print(f"Training data length: {len(train_data)}")
    print(f"Validation data length: {len(val_data)}")
    print(f"Testing data length: {len(test_data)}")
    
    return flatten_list(train_data), flatten_list(val_data), flatten_list(test_data)

In [8]:
# Dzielę dataset na dane testowe, walidacyjne i testowe. Następnie tworzę z nich dataset huggingface'owy
train, val, test = split_data(results, 0.7, 0.15, 0.15)
raw_prepared_dataset = data_to_dataset_dict(train, val, test)

Training data length: 4653
Validation data length: 997
Testing data length: 998


In [9]:
#Obliczam ilość par pozytywnych i negatywnych w datasecie, będzie mi to potrzebne do ustalenia wag
results_df = to_dataframe(flatten_list(results))
results_df.head()
positive_pairs_count = len(results_df[results_df["labels"] == 1])
negative_pairs_count = len(results_df[results_df["labels"] == 0])
all_pairs = len(results_df)
print(negative_pairs_count, positive_pairs_count, all_pairs)

128316 17110 145426


In [10]:
# Tokenizuję dataset, abym mógł użyć go do nauki modelu
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
    
tokenize_function = lambda example: tokenizer(example["query"], example["answer"], truncation=True, padding="max_length")
    
tokenized_datasets = raw_prepared_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["query", "answer"])
tokenized_datasets = tokenized_datasets.with_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/101573 [00:00<?, ? examples/s]

Map:   0%|          | 0/21926 [00:00<?, ? examples/s]

Map:   0%|          | 0/21927 [00:00<?, ? examples/s]

In [11]:
#Tworzę zmienne niezbędne do przeprowadzenia trenowania.
labels = np.array([0] * negative_pairs_count + [1] * positive_pairs_count)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

#weights_tensor służy do poinformowania modelu, że klasa 1(pary w których jest pytanie i odpowiedź na nie), powinna być traktowana priorytetowo
#ma to umożliwić poprawne trenowanie modelu, w sytuacji, gdy dataset jest niezrównoważony. Gdy nie ustawi się wag, model uczy się przypisywać wszystkim
#parom label 0, osiągając przy tym accuracy na poziomie 0.93, gdyż tak dużo jest par negatywnych.
weights_tensor = torch.tensor(class_weights, dtype=torch.float)
print(weights_tensor)

# Do trenowania używam klasy CustomTrainer, która dziedziczy po klasie Trainer. Wykonuję to, aby użyć innej niż standardowa funkcji kosztu.
# Zastosowałem funkcję "Focalloss, zamiast domyślnej "CrossEntropyLoss", korzystając z CrossEntropy nie mogłem doprowadzić do
# prawidłowego wytrenowania modelu. Poniżej prezentuję kod CustomTrainera z zastosowaniem funkcji CrossEntropy jako funkcji kosztu.
# Funkcja focal loss dobrze radzi sobie z niezbalansowanymi datasetami. Jej parametrami są: gamma i tensor wag klas.
# Gamma pozwala określić, jak bardzo model jest "karany" za błędne przydzielenie elementu do priorytetowej klasy. Przy dużym parametrze gamma,
# model był bardziej karany, za przydzielenie elementu o klasie 1 do klasy 0. 
# W implementacji wykorzystałem gotową implementację focal loss https://github.com/mathiaszinnen/focal_loss_torch
class CustomTrainer(Trainer):
    def __init__(self, *args, weights, gamma = 5.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.loss_fct = FocalLoss(gamma=gamma, weights=weights)


    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.loss_fct(self.softmax(logits.view(-1, self.model.config.num_labels)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Implementacja klasy Custom Trainer, wykorzystującej nn.CrossEntropyLoss. Pomimo moich usilnych prób(widać je we fragmencie, gdzie ustawiam wagi
# dla poszczególnych klas. Dla klasy 1, użyłem wagi będącej 3-krotnością stosunku par_pozytywnych do wszystkich par. Pomimo moich usilnych starań,
# postanowiłem porzucić tą funkcję kosztu. W późniejszej analizie użyłem jednego z modeli wytrenowanych tą klasą trainer.
# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([positive_pairs_count/all_pairs, 3*negative_pairs_count/all_pairs], device=model.device))
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

weights_tensor = weights_tensor.to('cuda')

# Tworzę argumenty treningowe. W trakcie testów okazało się, że pozytywnie na wynik wpływa parametr "gradient_accumulation_steps". Podejrzewam,
# że w batchu wielkości 5, zbyt rzadko występowały pary pozytywne, co skutkowało gorszym trenowaniem. Natomiast przy kumulowaniu błędów z kilku batchy
# mamy pewność, że praktycznie zawsze na zmiany w modelu będzie miał wpływ co najmniej jeden przykład pozytywny.
training_args = TrainingArguments(
    output_dir='./results7',
    num_train_epochs=3,
    per_device_train_batch_size=5,
    logging_dir='./logs',
    gradient_accumulation_steps=6,
    evaluation_strategy="epoch",
    run_name="run-test1",
    warmup_steps=600,
    save_strategy = "epoch",
)

tensor([0.5667, 4.2497])


In [12]:
# Przygotowuję model, ktory bedzie trenowany. W ramach laboratorium, korzystałem zarówno z modelu "bert-base-cased", jak i dotrenowywałem swoje
# modele, na przykład gdy ich trening został przerwany.
#source = "./results5/checkpoint-10787"

model_to_be_trained = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
#model_to_be_trained = AutoModelForSequenceClassification.from_pretrained(source)

trainer = CustomTrainer(
    model_to_be_trained,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    weights=weights_tensor,
)
#trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ładuję wytrenowany wcześniej model, który daje najlepsze rezultaty

In [13]:
#Przygotowuję funkcje do wczytywania modelu i wypisywania metryk dla niego.

def get_trainer(source):
    model = AutoModelForSequenceClassification.from_pretrained(source)
    # Training args nie mają znaczenia. Abstrakcja trainer, jest używana jeydnie jako wygodny pojemnik na model, do wywołania funkcji
    # trainer.predict(Datset).
    training_args = TrainingArguments(output_dir="./results", per_device_eval_batch_size=5)
    trainer = Trainer(model,training_args)
    return trainer

def get_metrics_for_trainer(trainer, tokenized_dataset):
    predictions = trainer.predict(tokenized_datasets["test"])
    preds = np.argmax(predictions.predictions, axis=-1)

    accuracy_metric = evaluate.load("accuracy")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")
    mcc_metric = evaluate.load("matthews_correlation")
    
    results = {}
    results["accuracy"] = accuracy_metric.compute(predictions=preds, references=predictions.label_ids)["accuracy"]
    results["precision"] = precision_metric.compute(predictions=preds, references=predictions.label_ids)["precision"]
    results["recall"] = recall_metric.compute(predictions=preds, references=predictions.label_ids)["recall"]
    results["f1"] = f1_metric.compute(predictions=preds, references=predictions.label_ids)["f1"]
    results["mcc"] = mcc_metric.compute(predictions=preds, references=predictions.label_ids)["matthews_correlation"]
    print(results)

In [14]:
def count_ndcg5(ids_returned, ids_correct):
    DCG = 0
    for i in range(5):
        if ids_returned[i] in ids_correct:
            DCG += 1/math.log(i+2,2)
    IDCG = 0
    for i in range(min(len(ids_correct), 5)):
        IDCG += 1/math.log(i+2, 2)
    return DCG/IDCG

# Funkcja dokonuje zamiany pytania i zwróconych id z korpusu na dataset par pytanie odpowiedź, który będzie mógł zostać stokenizowany 
# i na którym bedzie mogła zostać przeprowadzona klasyfikacja przez model.
def get_answers_from_corpus(question, corpus_df, answers_id):
    result = []
    for answer_id in answers_id:
        value = corpus_df[corpus_df["_id"] == answer_id]["text"].values
        if len(value) == 0:
            print(elastic_answer)
        else:
            result.append([question, value[0]])
    return pd.DataFrame(result, columns=["query", "answer"])

# Funkcja do obliczenia NDCG dla konkretnego pytania, dla konkretnego modelu. Z elasticsearch, uzyskuję SIZE_FOR_MODEL id zdań, które najlepiej
# pasują do zadanego pytania. Następnie z uzyskanych pytań, tworzę pary z pytaniem, które potem zamienione na dataset huggingface, a
# następnie są tokenizowane. Po tokenizacji dokonuję klasyfikacji modelem, a na koniec przetwarzam wyniki i zwracam obliczone ndcg@5.
def search_best_answers_model(trainer, tokenizer, client, question, correct_answers, corpus_df, SIZE_FOR_MODEL):
    tokenize_function = lambda example: tokenizer(example["query"], example["answer"], truncation=True, padding="max_length")
    #uzyskiwanie odpowiedzi z elasticsearch
    proposed_answers = search_best_answers_es(client, "qa_index", question, SIZE_FOR_MODEL)
    #dataset pandasa z kolumnami [query, answer], który zostanie potem zamieniony na dataset huggingface
    pairs = get_answers_from_corpus(question, corpus_df, proposed_answers)
    model_input = Dataset.from_pandas(pairs)
    #tokenizacja
    tokenized_model_input = model_input.map(tokenize_function)
    #model klasyfikuje pary
    results = trainer.predict(tokenized_model_input)
    #liczę softmax z predykcji modelu, aby dostać prawdopodobieństwo należenia pary do pozytynej grupy.
    results = softmax(results.predictions, axis = 1)
    results_with_ids = list(zip(results, proposed_answers))
    #sortuję wyniki, aby być w stanie otrzymać pierwszych 5 elementów
    results_with_ids.sort(key=lambda x: x[0][1], reverse = True)
    chosen_elements = [element[1] for element in results_with_ids][:5]
    # na podstawie pierwszych 5 elementów, liczę ndcg i wynik zwracam
    return count_ndcg5(chosen_elements, correct_answers)

#Funkcja do przeprowadzenia pomiarów dla konkretnego modelu. Więcej sczegółów w komentarzu do funkcji search_best_answers_model
def perform_experiment_for_model(trainer, tokenizer, client, corpus_df, qa_dataset_test, dataset_questions, SIZE_FOR_MODEL):
    results = []
    ids = qa_dataset_test["query-id"].unique()
    #iterations = 1
    iterations = len(ids)
    for i in tqdm(range(iterations), desc="Processing"):
        id = ids[i]
        question = dataset_questions[dataset_questions['_id'] == id]["text"].to_list()[0]
        res = search_best_answers_model(trainer, tokenizer, client, question, qa_dataset_test[qa_dataset_test["query-id"] == id]["corpus-id"].values, corpus_df, SIZE_FOR_MODEL)
        results.append(res)
    return results

In [15]:
SIZE_FOR_MODEL = 100
# Wczytuję najlepszy model, aby policzyć dla niego metryki i przeprowadzić eksperymenty.
all_results = []
source = "./work/checkpoint-10787"
trainer = get_trainer(source)
#get_metrics_for_trainer(trainer, tokenized_datasets)
all_results.append(perform_experiment_for_model(trainer, tokenizer, client, corpus_df, qa_dataset_test, dataset_questions, SIZE_FOR_MODEL))

{'accuracy': 0.869019929766954, 'precision': 0.46597679523361557, 'recall': 0.5596986817325801, 'f1': 0.5085557837097879, 'mcc': 0.4361479485537518}


Processing:   0%|          | 0/648 [00:00<?, ?it/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
#NDCG@5 osiągnięte przez model.
print(np.mean(np.array(all_results[0])))

0.14473247618255872


Powyżej prezentuję wyniki osiągniete dla najlepszego modelu wytrenowanego przy pomocy funkcji Focal Loss.

In [17]:
# Wczytuję najlepszy model, aby policzyć dla niego metryki i przeprowadzić eksperymenty.
source = "./MODEL_PON_RANO"
trainer = get_trainer(source)
get_metrics_for_trainer(trainer, tokenized_datasets)
all_results.append(perform_experiment_for_model(trainer, tokenizer, client, corpus_df, qa_dataset_test, dataset_questions, SIZE_FOR_MODEL))

{'accuracy': 0.878916404432891, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'mcc': 0.0}


Processing:   0%|          | 0/648 [00:00<?, ?it/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
#NDCG@5 osiągnięte przez model.
print(np.mean(np.array(all_results[1])))

0.026793369787157895


Powyżej prezentuję wyniki osiągniete dla najlepszego modelu wytrenowanego przy pomocy funkcji Cross Entropy Loss.

## Do you think simpler methods, like Bayesian bag-of-words model, would work for sentence-pair classification? Justify your answer.

Uważam, że każda metoda ma swoje mocne i słabe strony, które powoduja, że nadaje się do użycia w konkretnych przypadkach. Proste metody, takie jak Bayesian bag-of-words bierze pod uwagę jedynie występowanie konkretnych słów w tekście, natomiast metoda ta ignoruje kontekst i znaczenie słów w tekście. Konsekwencją prostoty jest niski koszt ewaluacji, a także gorsze wyniki niż osiągane dla modeli opartych na sieciach neuronowych. 

Istnieją jednak sytuacje, gdy prostota i szybkość działania ma duże znaczenie. W takich sytuacjach wykorzystanie sieci neuronowych jest niemożliwe, więc wybiera się meteody prostsze, szybsze, godząc się na większą liczbę błędów w wynikach.

## What hyper-parameters you have selected for the training? What resources (papers, tutorial) you have consulted to select these hyper-parameters?

Najważniejszymi hiper-parametrami, wykorzystanymi w pracy nad trenowaniem modelu, jest funkcja kosztu. W pracy nad trenowaniem modelu opierałem się na tutorialu stworzonym prze huggingface. Zastosowane tam podejście zupełnie nie pasowało do niezbalansowanego modelu, którym dysponowałem. Zacząłem więc drążyć temat w poszukiwaniu odpowiedzi na pytanie, jak wytrenować model gdy dataset jest niezbalansowany w taki sposób, że jedna klasa stanowi około 90% przykładów treningowych.
Pierwszy raz o focal-loss dowiedziałem się z pomocą chatu-gpt, który wskazał ją jako jeden ze sposóbów wytrenowania modelu pomimo niezbalansowanego datasetu. Zainteresowałem się tematem i znalazłem artykuły, wskazujące na to, że focal loss może być odpowiedzią na moje problemy:

- https://towardsdatascience.com/focal-loss-a-better-alternative-for-cross-entropy-1d073d92d075
- https://amaarora.github.io/posts/2020-06-29-FocalLoss.html
- https://rmoklesur.medium.com/reasons-to-choose-focal-loss-over-cross-entropy-5fdccb25d282

Udało mi się znaleźć gotową implementację focal loss w pytorch: 

https://github.com/mathiaszinnen/focal_loss_torch

---

Kolejnymi ważnymi hiperparametrami użytymi podczas treningu jest batch_size i gradient_accumulation_steps. Ponieważ moja karta graficzna dysponuje jedynie 6GB VRAM, największy batch-size na jaki mogłem sobie pozwolić, wynosi jedynie 5. Jeżeli pary negatywne stanowią około 90% par w całym datasecie, oznaczało to, że podczas uczenia modelu, często dochodziło do sytuacji, gdy był on uczony jedynie na negatywnych przykładach. Pomocny w tym przypadku okazał się być parametr "gradient_accumulation_steps", dzięki któremu praktycznie zawsze miałem pewność, że w danych, które wpływają na zmiany w modelu znjadowała się przynajmniej jedna para pozytywna.

## Think about pros and cons of the neural-network models with respect to natural language processing. Provide at least 2 pros and 2 cons.

Zastosowanie sieci neuronowych daje bardzo duże możliwość podczas tworzenia modeli językowych. Według mnie pozytywne aspekty ich użycia zdecydowanie przeważają nad negatywnymi. Warto w tymi miejscu jednak omówić po krótce argumenty które pokazują zarówno dobrem, jak i złe strony użycia sieci neuronowych.

Negatywne:
- **Trenowanie sieci neuronowych zajmuje bardzo dużo czasu i wymaga posiadania sporej ilosci zasobów.** Niestety nie da się nic na to poradzić. Do wytrenowania sieci potrzeba dużej ilości danych i dużo czasu. Jednak dysponując potężniejszym sprzętem i komputer osobisty, można to wykonać co udowodniła firma "openAI" tworząc modele GPT.
- **Wykonywanie predykcji równieża zajmuje dużo czasu.** W moim przypadku konieczne było wsparcie się elasticsearch'em do wykonania zadania. Jednak warto zauważyć, że dana technologia nie musi być wykorzystywana samodzielnie. Umiejętność łączenia narzędzi zgodnie z ich słabymi i mocnymi stronami umożliwia nam wykorzystanie ich w najlepszy możliwy sposób. Tak więc ta negatywna strona modeli językowych może być bardzo prosto ograniczona.
- **Modele oparte o sieci neuronowe są bardzo trudne do przeanalizowania. Tak naprawdę są czarną skrzynką, która zwraca wyniki i bardzo ciężko stwierdzić, dlaczego zostały zwrócone akurat takie wyniki.** Istnieją narzędzia umożliwiające analizę stworzonych sieci neuronowych. Jednak w dalszym ciągu jest to operacja trudna.

Pozytywne:
- **Pozwalają dokonać modelowania języka na bardzo dobrym poziomie. Modele takie jak GPT-4, pozwalają nam odbywać z nimi rozmowy, a nawet pomagają nam w codziennych zadaniach.** Bez sieci neuronowych nie bylibyśmy w stanie na takim poziomie zamodelować język.
- **Pozwalają bez wiedzy lingwistycznej dokonać modelowania języka, tak naprawdę same go analizując, programista musi jedynie dostarczyć dane.** Nawet ja, nie mając wiedzy o języku, byłem w stanie wytrenować model do zadania klasyfikacji zdań i odpowiedzi.

Podsumowując argumenty za i przeciw wykorzystaniu sieci neuronowych, wydaje mi się, że pomimo faktu, że jest ich mniej, argumenty pozytywne zdecydowanie przeważają nad argumentami negatywnymi. Oznacza to, że sieci neuronowe są bardzo dobrym narzędziem do używania podczas modelowania języka.